#### Optimal Blending of Oil Crudes

In [151]:
using JuMP, Clp, Plots,Printf

In [152]:
crude_blending = Model(solver=ClpSolver());

In [153]:
price_c1        = 35    # Cost of crude 1  [$/bbl]
price_c2        = 25    # Cost of crude 2  [$/bbl]
price_c3        = 55    # Cost of crude 3  [$/bbl]

pc_c1           = 1.6     # Crude 1 processing cost [$/bbl]
pc_c2           = 1.4  # Crude 2 processing cost [$/bbl]
pc_c3           = 1.0  # Crude 3 processing cost [$/bbl]

price_gasoline  = 45 # Cost of gasoline [$/bbl]
price_kerosene  = 37 # Cost of kerosene [$/bbl]
price_jetfuel   = 30 # Cost of jetfuel  [$/bbl]
price_residual  = 18 # Cost of residual [$/bbl]

Dgasoline       = 35000 # Gasoline demand [bbl/day]
Dkerosene       = 7000  # Gasoline demand [bbl/day]
Djetfuel        = 10000 # Gasoline demand [bbl/day]
Dresidual       = 5000 # Gasoline demand [bbl/day];

Acrude1         = 35000; # Crude 1 amount available [bbl/day]
Acrude2         = 45000; # Crude 2 amount available [bbl/day]
Acrude3         = 40000; # Crude 3 amount available [bbl/day]

In [154]:
@variable(crude_blending, bcrude1   >= 0, start = 10000)  # [bbl/day]
@variable(crude_blending, bcrude2   >= 0, start = 10000)  # [bbl/day]
@variable(crude_blending, bcrude3   >= 0, start = 10000)  # [bbl/day]
@variable(crude_blending, bgasoline >= 0, start = 10000)  # [bbl/day]
@variable(crude_blending, bkerosene >= 0, start = 10000)  # [bbl/day]
@variable(crude_blending, bjetfuel  >= 0, start = 10000)  # [bbl/day]
@variable(crude_blending, bresidual >= 0, start = 10000)  # [bbl/day];

#### Amount of manufactured products [bbl/day]

In [155]:
@constraint(crude_blending, c1, bgasoline == 0.75bcrude1 + 0.50bcrude2 + 0.60*bcrude3)
@constraint(crude_blending, c2, bkerosene == 0.10bcrude1 + 0.15bcrude2 + 0.18*bcrude3)
@constraint(crude_blending, c3, bjetfuel  == 0.10bcrude1 + 0.25bcrude2 + 0.20*bcrude3)
@constraint(crude_blending, c4, bresidual == 0.05bcrude1 + 0.10bcrude2 + 0.12*bcrude3);

#### Meet products demand

In [156]:
@constraint(crude_blending, c5, bgasoline <= Dgasoline)
@constraint(crude_blending, c6, bkerosene >= Dkerosene)
@constraint(crude_blending, c7, bjetfuel  >= Djetfuel)
@constraint(crude_blending, c8, bresidual >= Dresidual);

#### The amount of deployed crude should be smaller than the available supply

In [157]:
@constraint(crude_blending, c9,  bcrude1 <= Acrude1)
@constraint(crude_blending, c10, bcrude2 <= Acrude2)
@constraint(crude_blending, c11, bcrude3 <= Acrude3);

#### Objective Function: Maximize Profits

In [158]:
@objective(crude_blending, Max, price_gasoline*bgasoline + price_kerosene*bkerosene + price_jetfuel*bjetfuel +
                           price_residual*bresidual - price_c1*bcrude1 - price_c2*bcrude2 - price_c3*bcrude3 - 
                           pc_c1*bcrude1 - pc_c2*bcrude2 - pc_c3*bcrude3);

In [159]:
status_crude_blending = solve(crude_blending)

:Optimal

In [160]:
println("Status of the problem is: ", status_crude_blending)
println("Objective function value [\$/day] = ", getobjectivevalue(crude_blending))
println("---------------------------------------")
println(" Raw Material [bbl/day]")
@printf " Crude No. 1: %f\n" getvalue(bcrude1)
@printf " Crude No. 2: %f\n" getvalue(bcrude2)
@printf " Crude No. 3: %f\n" getvalue(bcrude3)
println("---------------------------------------")
println(" Products [bbl/day]         Demand [bbl/day]")
@printf " Gasoline: %8.2f            %8.2f\n" getvalue(bgasoline)  Dgasoline
@printf " Keronese: %8.2f            %8.2f\n" getvalue(bkerosene)  Dkerosene
@printf " Jet Fuel: %8.2f            %8.2f\n" getvalue(bjetfuel)   Djetfuel
@printf " Residual: %8.2f            %8.2f\n" getvalue(bresidual)  Dresidual
println("---------------------------------------")


Status of the problem is: Optimal
Objective function value [$/day] = 571916.6666666669
---------------------------------------
 Raw Material [bbl/day]
 Crude No. 1: 16666.666667
 Crude No. 2: 45000.000000
 Crude No. 3: 0.000000
---------------------------------------
 Products [bbl/day]         Demand [bbl/day]
 Gasoline: 35000.00            35000.00
 Keronese:  8416.67             7000.00
 Jet Fuel: 12916.67            10000.00
 Residual:  5333.33             5000.00
---------------------------------------
